<a href="https://colab.research.google.com/github/haxiaww/sentiment-prediction-app/blob/app/giao_dien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


##fundamentals

In [ ]:
## Thư viện
import imageio
import joblib            as jlb
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import pydotplus         as pdp
import seaborn           as sns

import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn                 import tree
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import string
punct=string.punctuation

In [ ]:
def remove(text):
  text=re.sub(r'[0-9]+','',text).lower()#xóa số
  text=re.sub(r'(www|http)\S+', '', text) #xóa URL
  text=text.translate(str.maketrans('','',punct)) #xóa punctuation
  return text

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def remove_stopwords(text):
  return " ".join([word for word in text.split() if word not in set(stopword_list)])

In [ ]:
def abbreviation(text):
  #Điều này có nghĩa là nó coi trích dẫn đơn là một ký tự chữ chứ không phải là
  #một ký tự đặc biệt với ý nghĩa thông thường trong biểu thức chính quy
  text=re.sub(r'can\'t','can not',text)
  text=re.sub(r'n\'t','not',text)
  text=re.sub(r'\'m','can not',text)
  text=re.sub(r'\'ll','will',text)
  text=re.sub(r'\'ve','have',text)
  text=re.sub(r'\'t','not',text)
  return text

In [ ]:
def space(text):
  text = re.sub('\s\s+',' ',text)
  return text #xóa khoảng trắng trùng lắp

In [ ]:
def stem_and_lemmatize(text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatized_words)

##main interface

In [111]:
@anvil.server.callable
def cleaned_text(text):
  text=abbreviation(text)
  text=remove(text)
  text=remove_stopwords(text)
  text=space(text)
  text=stem_and_lemmatize(text)
  return(text)

In [107]:
from getpass import getpass
uplink_key = getpass('Enter your Uplink key: ')

Enter your Uplink key: ··········


In [108]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=883f3e68b512a24ca3534ccf66c8dd4cd0744b7c520e2f4451d67ce32932b48c
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [109]:
import anvil.server

In [110]:
anvil.server.connect(uplink_key)

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [79]:
import pickle
from tensorflow.keras.models import load_model

In [120]:
with open('/content/gdrive/My Drive/Colab Notebooks/Data/tf_idf.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

In [163]:
ann_model_filepath = '/content/gdrive/My Drive/Colab Notebooks/dl_model.pkl'
with open(ann_model_filepath, 'rb') as file:
    ann_model = pickle.load(file)
log_model_filepath = '/content/gdrive/My Drive/Colab Notebooks/logit_model.pkl'
with open(log_model_filepath, 'rb') as file:
    log_model = pickle.load(file)
tree_model_filepath = '/content/gdrive/My Drive/Colab Notebooks/tree_model.pkl'
with open(tree_model_filepath, 'rb') as file:
    tree_model = pickle.load(file)

In [166]:
@anvil.server.callable
def predict_sentiment_logistic(input_text):
    preprocessed_text = cleaned_text(input_text)
    input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
    prediction = log_model.predict([input_vector])
    if prediction[0] >= 0.5:
        sentiment = 'Positive'
    else:
        sentiment = 'Negative'
    res = "Model Logistic Regression predicted the input text to be: " + sentiment
    return res
@anvil.server.callable
def predict_sentiment_tree(input_text):
    preprocessed_text = cleaned_text(input_text)
    input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
    prediction = tree_model.predict([input_vector])
    if prediction[0] >= 0.5:
        sentiment = 'Positive'
    else:
        sentiment = 'Negative'
    res = "Model Decision Tree predicted the input text to be: " + sentiment
    return res
@anvil.server.callable
def predict_sentiment_ann(input_text):
    preprocessed_text = cleaned_text(input_text)
    input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
    prediction = ann_model.predict([input_vector])
    if prediction[0] >= 0.5:
        sentiment = 'Positive'
    else:
        sentiment = 'Negative'
    res = "Model ANN predicted the input text to be: " + sentiment
    return res

In [164]:
@anvil.server.callable
def logistic_predicted_score(input_text):
  preprocessed_text = cleaned_text(input_text)
  input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
  prediction = log_model.predict([input_vector])
  res = "Model Logistic Regression predicted sentiment score: " + str(prediction[0])
  return res
@anvil.server.callable
def tree_predicted_score(input_text):
  preprocessed_text = cleaned_text(input_text)
  input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
  prediction = tree_model.predict([input_vector])
  res = "Model Decision Tree predicted sentiment score: " + str(prediction[0])
  return res
@anvil.server.callable
def ann_predicted_score(input_text):
  preprocessed_text = cleaned_text(input_text)
  input_vector = tfidf_vectorizer.transform([preprocessed_text]).toarray()
  prediction = ann_model.predict([input_vector])
  res = "Model ANN predicted sentiment score: " + str(prediction[0])
  return res

In [165]:
@anvil.server.callable
def final_report(input_text):
  res1 = logistic_predicted_score(input_text)
  res2 = tree_predicted_score(input_text)
  res3 = ann_predicted_score(input_text)
  res = res1 + '\n' + res2 + '\n' + res3
  return res

In [ ]:
anvil.server.wait_forever()